In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</s}tyle>"))

In [2]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from pathlib import Path
import glob
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import random
import pickle
import os
import config
import random
from natsort import natsorted
import lfp
arm = 'UR5'


print('Using local setup')
WORKING_PATH = Path().absolute().parent
print(f'Working path: {WORKING_PATH}')
os.chdir(WORKING_PATH)
STORAGE_PATH = WORKING_PATH
print(f'Storage path: {STORAGE_PATH}')
UNITY = True


Using local setup
Working path: /catkin_ws/src/robotics/learning_from_play
Storage path: /catkin_ws/src/robotics/learning_from_play


In [3]:

import lfp.unity_utils

In [4]:
from lfp.data import serialise_vid, read_vid

In [5]:

import rospy
from robotics_demo.msg import JointPositions, Observation, AchievedGoal, PositionCommand, RPYProprioState, Velocities, ResetInfo, ToRecord, RPYState, Reengage,  QuaternionProprioState,  Goal, TimerBeat
from robotics_demo.srv import getIK, getIKResponse, getState, getStateResponse, getTime, getTimeResponse
import time



    
def process_observation_srv():
    '''
    The full state will be sent out at Nh > controlHz by the env, listen to it, and save the relevant parts
    '''
    o = getStateServ(int(1)).state
    proprioceptive_state  = proprio_quat_to_rpy_vector(o.proprio)
    achieved_goal = ag_to_vector(o.ag)
    full_state = np.concatenate([proprioceptive_state, achieved_goal])
    # process the images
    o.shoulderImage.data += (o.imq2 + o.imq3 + o.imq4)
    ros_shoulder_image = o.shoulderImage
    ros_gripper_image = o.gripperImage
    shoulder_image  = rosImg_to_numpy(ros_shoulder_image)[:,:,:3]
    gripper_image = rosImg_to_numpy(ros_gripper_image)[:,:,:3]
    velocities = o.vels
    gen_time = o.time
    return proprioceptive_state, achieved_goal, shoulder_image, gripper_image, velocities, gen_time


def check_reset_convergence(proprio, ag, current_proprio, current_ag, threshold = 0.01):
    proprio_checks = abs(proprio - current_proprio) < threshold
    ag_checks = abs(ag-current_ag) < threshold
    all_checks = np.concatenate([proprio_checks, ag_checks])
    return all_checks
#     if np.all(all_checks):
#         return True
#     else:
#         return False
    
def converge_on_reset(proprio, ag, threshold = 0.05):
    t_start = time.time()
    r = rospy.Rate(10) # 10hz 
    while not rospy.is_shutdown():
        t = time.time()
        current_arm, current_ag, _,_, _,_ = process_observation_srv()
        checks = check_reset_convergence(proprio, ag, current_arm, current_ag, threshold)
        if (t > t_start+0.01):
            idxs = np.where(checks == False)
#             print(idxs, proprio[idxs], current_arm[idxs])
            return
        elif np.all(checks):
            return
        r.sleep() # ros sleep for a little while before checking convergence again
        
        


def reset_to(arm, ag, prev_act = np.zeros(7)):
    
    arm_msg = proprio_rpy_to_ROSmsg(arm)
    #prev_act = act_to_jointPositionsROSmsg(prev_act)
    # Get IK for actual position
    prev_act = JointPositions()
    
    
    # Package into ResetInfo
    for i in range(0,2):
        joints = IK(arm_msg, prev_act).joints # once to align it with the prev acts
        r = ResetInfo(joints, ag_to_ROSmsg(ag), Velocities())
        # Send down
        env_reset_pub.publish(r)
    converge_on_reset(arm, ag, threshold = 0.05)
    env_reEngage_collision_pub.publish(Reengage())
# The publisher which sends out position commands that then get converted to joint commands by the IK node
pos_cmd_pub = rospy.Publisher('xyz_rpy_g_command', PositionCommand, queue_size=1)
# The publisher which resets the non_arm elements of the state
env_reset_pub = rospy.Publisher('full_reset', ResetInfo, queue_size=1)
env_reEngage_physics_pub = rospy.Publisher('re_engage_physics', Reengage, queue_size=1)
env_reEngage_collision_pub = rospy.Publisher('re_engage_collision', Reengage, queue_size=1)
toggleTimePub = rospy.Publisher('toggleHalt', Reengage, queue_size=1)
# The publisher which sends out the consolidated state for the saver to save
transition_pub = rospy.Publisher('timestep', ToRecord, queue_size=1)
# service to get IK 
IK = rospy.ServiceProxy('get_IK', getIK)
getStateServ = rospy.ServiceProxy('getState', getState, persistent=True)
syncTimeServ = rospy.ServiceProxy('getTime', getTime, persistent=True)


rospy.init_node('data_maker')

In [6]:


import tensorflow_hub as hub
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# model = hub.load(module_url)
# print ("module %s loaded" % module_url)
# def embed(input):
#   return model(input)


embed = hub.KerasLayer(str(STORAGE_PATH)+'/saved_models/universal_sentence_encoder')

In [7]:
if UNITY:
    TRAIN_DATA_PATHS = [STORAGE_PATH/'data'/x for x in ["diverse"]]
    TEST_DATASET = "diverse_test" 

TEST_DATA_PATH = STORAGE_PATH/'data'/TEST_DATASET

In [8]:
path =  TRAIN_DATA_PATHS[0] # TEST_DATA_PATH #   
keys = ['obs','acts','achieved_goals']
obs_act_path = os.path.join(path, 'obs_act_etc/')
directories = natsorted(os.listdir(obs_act_path))
print(path)

filename = 'labels'
tagging = False
if tagging: filename = 'tagged_labels' 

/catkin_ws/src/robotics/learning_from_play/data/diverse


In [9]:
indices_and_labels = list(np.load(path/f'{filename}.npz', allow_pickle=True)['labels'])
indices_and_labels[0]

{'indices': {'traj': 19, 'start': 9345, 'end': 9380},
 'label': 'grasp the blue block',
 'tag': ''}

In [10]:
from lfp.unity_utils.messaging import rosImg_to_numpy, proprio_quat_to_rpy_vector, \
        proprio_rpy_to_ROSmsg, ag_to_vector, ag_to_ROSmsg,\
        proprio_rpy_to_rpy_vector, unstack, act_to_jointPositionsROSmsg

from IPython.display import display, clear_output
import matplotlib.pyplot as plt

sub_dict = {'red': 'yellow', 'green':'black', 'blue':'purple'}

def get_dataset(directories, index=None):
    if index == None:
        demo = random.choice(directories)
    else:
        demo = str(index)
    dataset = {k:[] for k in keys+['sequence_index','sequence_id','im_path']}
    traj = np.load(obs_act_path+demo+'/data.npz')
    for k in keys:
        d = traj[k]
        if len(d.shape) < 2:
            d = np.expand_dims(d, axis = 1) # was N, should be N,1
        dataset[k] = d.astype(np.float32)
    timesteps = len(traj['obs'])
    dataset['sequence_index']= np.arange(timesteps, dtype=np.int32).reshape(-1, 1)
    dataset['sequence_id'] = np.full(timesteps, fill_value=int(demo), dtype=np.int32).reshape(-1, 1)
    dataset['im_path'] = [str(path) for i in range(0,timesteps)]
    dataset['shoulder_im_path'] = [str(s) + '/states_and_ims/'+str(dataset['sequence_id'][i][0]) + '/ims/'+str(dataset['sequence_index'][i][0])+'_shoulder.jpg' for i,s in enumerate(dataset['im_path'])]
    dataset['gripper_im_path'] = [str(s) + '/states_and_ims/'+str(dataset['sequence_id'][i][0]) + '/ims/'+str(dataset['sequence_index'][i][0])+'_gripper.jpg' for i,s in enumerate(dataset['im_path'])]
    return dataset


def make_vid(i, indices_and_labels, directories, max_len=50, shoulder_im_dim=128):
    
    indices, label, tag = indices_and_labels[i]['indices'], indices_and_labels[i]['label'], indices_and_labels[i]['tag'] 
    
    traj, start, end = indices['traj'], indices['start'], indices['end']
    dataset = get_dataset(directories, traj)
    # obs (end-start, 35) 
    obs = dataset['obs'][start:end]
    # goals (end-start, 28) 
    ags = dataset['achieved_goals'][start:end]
    
    
    # seq_lens (1,)
    seq_lens = end-start
    # dataset_path (end-start)
    dataset_path = np.array([traj]*seq_lens)
    # tstep_idxs (end-start)
    tsep_idxs = np.arange(start, end)
    
    ims = []
    
    for j in range(0, seq_lens):
        proprio = obs[:, :7]
        reset_to(proprio[j,:], ags[j,:])
        #clear_output(wait=True)
        proprioceptive_state, achieved_goal, shoulder_image, gripper_image, velocities, gen_time = process_observation_srv()
        #plt.imshow(shoulder_image)
        ims.append(shoulder_image)
        #plt.show()

    # imgs (end-start, 128, 128, 3)
    shoulder_imgs = np.concatenate([ims],0)

    # (128, 128, 128)
    goal_imgs = shoulder_imgs[-1]

    masks = tf.ones([end-start])

    def pad(element, max_len, seq_len):
        padding = [[0,0] for i in range(0,len(element.shape))] # create padding the same dim as the tensor
        padding[0] = [0,max_len-seq_len] # create padding for the time dimension
        return tf.pad(element, padding) # pad and add a new axis

    trajectory = {'masks':masks,
                  'imgs':shoulder_imgs}

    trajectory = {k:pad(v, max_len, seq_lens) for k,v in trajectory.items()}

    trajectory['goal_imgs'] = goal_imgs
    trajectory['seq_lens'] = seq_lens
    
    for key in sub_dict.keys():
        label = label.replace(key, sub_dict[key])
        
    print(label)
    
    trajectory['label'] = label
    trajectory['label_embedding'] = np.squeeze(embed([label])) # # 512
    trajectory['tag'] = tag
    
    return trajectory

In [11]:
VIDEO_PATH =  STORAGE_PATH/'data'/'contrastive_vids/'
VIDEO_PATH

PosixPath('/catkin_ws/src/robotics/learning_from_play/data/contrastive_vids')

In [12]:
t = make_vid(25, indices_and_labels, directories)

move the purple block into the cupboard


In [13]:
from tqdm import tqdm
save_path = str(VIDEO_PATH/'tf_records')+f"/labelled_videos2.tfrecords"
with tf.io.TFRecordWriter(save_path) as file_writer:
    print(save_path)
    for i in tqdm(range(504,len(indices_and_labels))):
        t = make_vid(i, indices_and_labels, directories)
        byte_stream = serialise_vid(t)
        file_writer.write(byte_stream)

  0%|          | 0/496 [00:00<?, ?it/s]

/catkin_ws/src/robotics/learning_from_play/data/contrastive_vids/tf_records/labelled_videos2.tfrecords


  0%|          | 1/496 [00:05<42:24,  5.14s/it]

take the purple block out of the drawer


  0%|          | 2/496 [00:09<39:15,  4.77s/it]

...

close the drawer


100%|█████████▉| 495/496 [40:52<00:04,  4.69s/it]

pick up both blocks


100%|██████████| 496/496 [40:57<00:00,  4.95s/it]

press the yellow button


In [ ]:
gsutil cp data/contrastive_vids/tf_records/labelled_videos.tfrecords gs://lfp_europe_west4_a/data/Unity/contrastive_vids/tf_records/